In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import torch

import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

# specify GPU
device = torch.device("cuda")

In [ ]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

df_fake["class"] = 0
df_true["class"] = 1


with open("/content/New Task.csv", encoding="utf8", errors='ignore') as f:
  newtest= pd.read_csv(f)

In [ ]:

newtest.reset_index(inplace = True)
df_fake

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [ ]:

newtest = newtest.drop(["Link_Of_News", "Source","Date","Stated_On"], axis = 1)
newtest['Label'] = newtest['Label'].replace({'FALSE': 0})
newtest['Label'] = newtest['Label'].replace({'pants-fire': 0})
newtest['Label'] = newtest['Label'].replace({'barely-true':0})
newtest['Label'] = newtest['Label'].replace({'TRUE': 1})
newtest['Label'] = newtest['Label'].replace({'half-true': 0})
newtest['Label'] = newtest['Label'].replace({'mostly-true': 1})
# newtest['Label'] = newtest['Label'].replace({'full-flop': 0})
newtest = newtest.set_index("Label")
newtest = newtest.drop("half-flip", axis=0)

newtest = newtest.drop("no-flip", axis=0)

newtest = newtest.drop("full-flop", axis=0)
# newtest['Label'] = newtest['Label'].drop(['half-flip'],axis=0)
# newtest['Label'] = newtest['Label'].drop({'no-flip'})
newtest.reset_index(inplace = True)
newtest.head(50

             )

,Label,index,News_Headline
0,0,0,Says Osama bin Laden endorsed Joe Biden
1,0,1,CNN aired a video of a toddler running away fr...
2,0,2,Says Tim Tebow kneeled in protest of abortion ...
3,0,3,Even so-called moderate Democrats like Joe Bid...
4,1,4,"""Our health department, our city and our count..."
5,0,5,Says before he planned a rally on June 19 nobo...
6,0,6,Californias registered independent voters will...
7,0,7,Antifa now banging on residents doors in Seatt...
8,0,8,President Obama and Vice President Biden never...
9,0,9,Says these elite figures are on house arrest w...


In [ ]:
# with open("/content/IFND.csv", encoding="utf8", errors='ignore') as f:
#   ifnd= pd.read_csv(f)
# ifnd=ifnd.drop(["id","Image","Web","Category","Date"],axis=1)
# ifnd['Label'] = ifnd['Label'].replace({'TRUE': 1})
# ifnd['Label'] = ifnd['Label'].replace({'Fake': 0})
# ifnd = ifnd.sample(frac = 1)
# ifnd.reset_index(inplace = True)
# ifnd.rename(columns={'Statement': 'text'}, inplace=True)
# ifnd.rename(columns={'Label': 'label'}, inplace=True)
# ifnd.drop(["index"], axis = 1, inplace = True)
# ifnd.head(10)


In [ ]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470,-1):
    df_fake.drop([i], axis = 0, inplace = True)
df_true_manual_testing = df_true.tail(10)
for i in range(21416,21406,-1):
    df_true.drop([i], axis = 0, inplace = True)

In [ ]:
df_fake_manual_testing["class"] = 0
df_true_manual_testing["class"] = 1
df_marge = pd.concat([df_fake, df_true], axis =0 )
df = df_marge.drop(["title", "subject","date"], axis = 1)
df.isnull().sum()
df = df.sample(frac = 1)
df.reset_index(inplace = True)
df.rename(columns={'class': 'label'}, inplace=True)
df.drop(["index"], axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
with open("/content/redditoutput.csv", encoding="utf8", errors='ignore') as f:
  rtest= pd.read_csv(f)
ttest=rtest
ttest = ttest.drop(["Title", "URL"], axis = 1)
ttest.reset_index(inplace = False)
ttest=ttest.drop(ttest.columns[0],axis=1)
rtest=rtest.drop(rtest.columns[0],axis=1)
rtest.reset_index(inplace = True)

rtest

,index,Title,URL,Text
0,0,Salman Rushdie attack: 'The Satanic Verses' au...,https://www.opindia.com/2022/08/salman-rushdie...,Rushdie was attacked when he was about to giv...
1,1,Former minister and Kerala MLA KT Jaleel has s...,https://newsable.asianetnews.com/india/cpi-m-k...,Former minister and Kerala MLA KT Jaleel has ...
2,2,Salman Rushdie Stabbed On Stage At New York Event,https://www.ndtv.com/world-news/author-salman-...,Salman Rushdie on the floor after the attack ...
3,3,Urvashi Rautela addresses person as 'RP' in re...,https://www.crictracker.com/urvashi-rautela-ad...,\n\n Vaishnavi\n Jr. Staff Writer\n\nUpdated ...
4,4,Author Salman Rushdie attacked on lecture stag...,https://apnews.com/article/salman-rushdie-atta...,"CHAUTAUQUA, N.Y. (AP) — Salman Rushdie, whose..."
5,5,Oldest Mother: An Indian Woman Made History By...,https://www.shutterbulky.com/oldest-mother-an-...,Daljinder Kaur became the oldest mother at ag...
6,6,Bangladeshi Muslims make flags for Har Ghar Ti...,https://www.siasat.com/bangladeshi-muslims-mak...,After making multiple disparaging statements ...
7,7,"As India marks its first 75 years, Gandhi is d...",https://www.washingtonpost.com/world/2022/08/1...,"HYDERABAD, India — When Indian screenwriter V..."
8,8,Ranveer Singh Called For Questioning By Mumbai...,https://www.ndtv.com/india-news/ranveer-singh-...,Ranveer Singh's photo series for Paper magazi...
9,9,Yogi Adityanath is backtracking on his promise...,https://www.newslaundry.com/2022/08/12/yogi-ad...,A chief minister’s promise does not need to b...


In [ ]:
rtest.drop(rtest.columns[0],axis=1)

,Title,URL,Text
0,Salman Rushdie attack: 'The Satanic Verses' au...,https://www.opindia.com/2022/08/salman-rushdie...,Rushdie was attacked when he was about to giv...
1,Former minister and Kerala MLA KT Jaleel has s...,https://newsable.asianetnews.com/india/cpi-m-k...,Former minister and Kerala MLA KT Jaleel has ...
2,Salman Rushdie Stabbed On Stage At New York Event,https://www.ndtv.com/world-news/author-salman-...,Salman Rushdie on the floor after the attack ...
3,Urvashi Rautela addresses person as 'RP' in re...,https://www.crictracker.com/urvashi-rautela-ad...,\n\n Vaishnavi\n Jr. Staff Writer\n\nUpdated ...
4,Author Salman Rushdie attacked on lecture stag...,https://apnews.com/article/salman-rushdie-atta...,"CHAUTAUQUA, N.Y. (AP) — Salman Rushdie, whose..."
5,Oldest Mother: An Indian Woman Made History By...,https://www.shutterbulky.com/oldest-mother-an-...,Daljinder Kaur became the oldest mother at ag...
6,Bangladeshi Muslims make flags for Har Ghar Ti...,https://www.siasat.com/bangladeshi-muslims-mak...,After making multiple disparaging statements ...
7,"As India marks its first 75 years, Gandhi is d...",https://www.washingtonpost.com/world/2022/08/1...,"HYDERABAD, India — When Indian screenwriter V..."
8,Ranveer Singh Called For Questioning By Mumbai...,https://www.ndtv.com/india-news/ranveer-singh-...,Ranveer Singh's photo series for Paper magazi...
9,Yogi Adityanath is backtracking on his promise...,https://www.newslaundry.com/2022/08/12/yogi-ad...,A chief minister’s promise does not need to b...


In [ ]:
import re
import string
def wordopt(text):
    text = text.lower()
    text = re.sub("\[.*?\]", '', text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
df["text"] = df["text"].apply(wordopt)
newtest["News_Headline"] = newtest["News_Headline"].apply(wordopt)

In [ ]:
newtest = newtest.sample(frac = 1)
newtest.reset_index(inplace = True)
newtest.drop(["index"], axis = 1, inplace = True)
newtest.rename(columns={'News_Headline': 'text'}, inplace=True)
newtest.rename(columns={'Label': 'label'}, inplace=True)
newtest.head(10)

,level_0,label,text
0,1553,0,the homicide rate in baltimore is significant...
1,5428,1,says hillary clinton and millennials have the ...
2,4198,0,russian mansions obama seized were meant to b...
3,3575,1,there are more immigrants in our country righ...
4,9045,0,i ve been a cop in lake county michigan sin...
5,4910,1,the federal government is prohibited from neg...
6,3358,0,congresswoman martha mcsally voted to let com...
7,304,0,michigan gov gretchen whitmer has just been s...
8,2068,0,a photograph shows beto orourke nearly nude wi...
9,2866,1,half of everyone thats here illegally in the...


In [ ]:
lol=pd.concat([df,newtest], axis =0 )
lol.head(10)

,text,label,level_0
0,tuesday night after president obama gave his ...,0,NaN
1,washington reuters hillary clinton s campa...,1,NaN
2,washington reuters u s and chinese diplom...,1,NaN
3,an atlanta democrat makes some very valid poin...,0,NaN
4,so a wannabe female bomber was a pre school te...,0,NaN
5,every day barack obama rubs his radical transf...,0,NaN
6,washington reuters president barack obama ...,1,NaN
7,jared fogle the man who lost a ton of weight ...,0,NaN
8,the cover ups and mistruths appear to be numer...,0,NaN
9,so much for the scotus not being political che...,0,NaN


In [ ]:
# check class distribution
# ifnd['label'].value_counts(normalize = True)

In [ ]:
train_text, temp_text, train_labels, temp_labels = train_test_split(lol['text'], lol['label'],
                                                                    random_state=2018,
                                                                    test_size=0.3,
                                                                    stratify=lol['label'])

# ntest_text, ntemp_text, ntest_labels, ntemp_labels = train_test_split(ttest['text'], ttest['label'],
#                                                                     random_state=2018,
#                                                                     test_size=0.3,
#                                                                      stratify=ttest['label'])

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels,
                                                                random_state=2018,
                                                                test_size=0.5,
                                                                stratify=temp_labels)

In [ ]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
max_seq_len = 25

In [ ]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)
ntokens_test = tokenizer.batch_encode_plus(
    ttest['Text'].tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# labels=ttest['label'].tolist()

In [ ]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

ntest_seq = torch.tensor(ntokens_test['input_ids'])
ntest_mask = torch.tensor(ntokens_test['attention_mask'])

# # temp_list = ntest_labels.tolist()
# # dictionary that maps integer to its string value
# label_dict = {}

# # list to store integer labels
# # int_labels = []

# # for i in range(len(temp_list)):
# #     label_dict[i] = temp_list[i]
# #     int_labels.append(i)

# ntest_y = torch.tensor(labels)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [ ]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):

      super(BERT_Arch, self).__init__()

      self.bert = bert

      # dropout layer
      self.dropout = nn.Dropout(0.5)

      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)

      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model
      _, cls_hs = self.bert(sent_id, attention_mask=mask,return_dict=False)

      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)

      # apply softmax activation
      x = self.softmax(x)

      return x

In [ ]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [ ]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-4)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_wts = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_labels),
                                        y = train_labels
                                    )


print(class_wts)

[0.89524722 1.13251552]


In [ ]:
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights)

# number of training epochs
epochs = 4

In [ ]:
# function to train the model
def train():

  model.train()

  total_loss, total_accuracy = 0, 0

  # empty list to save model predictions
  total_preds=[]

  # iterate over batches
  for step,batch in enumerate(train_dataloader):

    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]

    sent_id, mask, labels = batch

    # clear previously calculated gradients
    model.zero_grad()

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)

  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [ ]:
# function for evaluating the model
def evaluate():

  print("\nEvaluating...")

  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0

  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(val_dataloader):

    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:

      # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)

      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():

      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader)

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [ ]:
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):

    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    #train model
    train_loss, _ = train()

    #evaluate model
    valid_loss, _ = evaluate()

    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')

    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 4
  Batch    50  of  1,198.
  Batch   100  of  1,198.
  Batch   150  of  1,198.
  Batch   200  of  1,198.
  Batch   250  of  1,198.
  Batch   300  of  1,198.
  Batch   350  of  1,198.
  Batch   400  of  1,198.
  Batch   450  of  1,198.
  Batch   500  of  1,198.
  Batch   550  of  1,198.
  Batch   600  of  1,198.
  Batch   650  of  1,198.
  Batch   700  of  1,198.
  Batch   750  of  1,198.
  Batch   800  of  1,198.
  Batch   850  of  1,198.
  Batch   900  of  1,198.
  Batch   950  of  1,198.
  Batch 1,000  of  1,198.
  Batch 1,050  of  1,198.
  Batch 1,100  of  1,198.
  Batch 1,150  of  1,198.

Evaluating...
  Batch    50  of    257.
  Batch   100  of    257.
  Batch   150  of    257.
  Batch   200  of    257.
  Batch   250  of    257.

Training Loss: 0.368
Validation Loss: 0.251

 Epoch 2 / 4
  Batch    50  of  1,198.
  Batch   100  of  1,198.
  Batch   150  of  1,198.
  Batch   200  of  1,198.
  Batch   250  of  1,198.
  Batch   300  of  1,198.
  Batch   350  of  1,198.
  

In [ ]:
import time
from datetime import date, datetime, time
from babel.dates import format_date, format_datetime, format_time

In [ ]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
# get predictions for test data
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()

In [ ]:
preds

array([[-0.09722601, -2.3789363 ],
       [-0.29365504, -1.3685865 ],
       [-0.00800094, -4.8321867 ],
       ...,
       [-0.28440273, -1.3961972 ],
       [-0.29281375, -1.3710555 ],
       [-0.0830116 , -2.529994  ]], dtype=float32)

In [ ]:
# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      4585
           1       0.92      0.90      0.91      3625

    accuracy                           0.92      8210
   macro avg       0.92      0.92      0.92      8210
weighted avg       0.92      0.92      0.92      8210



In [ ]:
# confusion matrix
pd.crosstab(test_y, preds)

col_0,0,1
row_0,,
0,4316,269
1,352,3273


In [ ]:
with torch.no_grad():
  preds1 = model(ntest_seq.to(device), ntest_mask.to(device))
  preds1 = preds1.detach().cpu().numpy()
  preds1 = np.argmax(preds1, axis = 1)

  preds1=pd.DataFrame(preds1)


In [ ]:
preds1 = preds1.reset_index(drop=False)
preds1

,index,0
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [ ]:



preds1 = pd.merge(rtest, preds1,how='left',on=['index'])

preds1

,index,Title,URL,Text,0
0,0,Salman Rushdie attack: 'The Satanic Verses' au...,https://www.opindia.com/2022/08/salman-rushdie...,Rushdie was attacked when he was about to giv...,0
1,1,Former minister and Kerala MLA KT Jaleel has s...,https://newsable.asianetnews.com/india/cpi-m-k...,Former minister and Kerala MLA KT Jaleel has ...,0
2,2,Salman Rushdie Stabbed On Stage At New York Event,https://www.ndtv.com/world-news/author-salman-...,Salman Rushdie on the floor after the attack ...,0
3,3,Urvashi Rautela addresses person as 'RP' in re...,https://www.crictracker.com/urvashi-rautela-ad...,\n\n Vaishnavi\n Jr. Staff Writer\n\nUpdated ...,0
4,4,Author Salman Rushdie attacked on lecture stag...,https://apnews.com/article/salman-rushdie-atta...,"CHAUTAUQUA, N.Y. (AP) — Salman Rushdie, whose...",0
5,5,Oldest Mother: An Indian Woman Made History By...,https://www.shutterbulky.com/oldest-mother-an-...,Daljinder Kaur became the oldest mother at ag...,0
6,6,Bangladeshi Muslims make flags for Har Ghar Ti...,https://www.siasat.com/bangladeshi-muslims-mak...,After making multiple disparaging statements ...,0
7,7,"As India marks its first 75 years, Gandhi is d...",https://www.washingtonpost.com/world/2022/08/1...,"HYDERABAD, India — When Indian screenwriter V...",0
8,8,Ranveer Singh Called For Questioning By Mumbai...,https://www.ndtv.com/india-news/ranveer-singh-...,Ranveer Singh's photo series for Paper magazi...,0
9,9,Yogi Adityanath is backtracking on his promise...,https://www.newslaundry.com/2022/08/12/yogi-ad...,A chief minister’s promise does not need to b...,0


In [ ]:
preds1 = preds1.rename(columns={preds1.columns[4]: 'label'})


In [ ]:
print(preds1)

    index                                              Title  \
0       0  Salman Rushdie attack: 'The Satanic Verses' au...   
1       1  Former minister and Kerala MLA KT Jaleel has s...   
2       2  Salman Rushdie Stabbed On Stage At New York Event   
3       3  Urvashi Rautela addresses person as 'RP' in re...   
4       4  Author Salman Rushdie attacked on lecture stag...   
5       5  Oldest Mother: An Indian Woman Made History By...   
6       6  Bangladeshi Muslims make flags for Har Ghar Ti...   
7       7  As India marks its first 75 years, Gandhi is d...   
8       8  Ranveer Singh Called For Questioning By Mumbai...   
9       9  Yogi Adityanath is backtracking on his promise...   
10     10  This bill could be dangerous to public healthc...   
11     11      18% GST (Tax) On House Rent: See Who Will Pay   
12     12  RSS Changes Profile Pictures Of Social Media A...   
13     13  Man overcharged 20 rupees for India train tick...   
14     14  [2010]RSS workers attack TV c

In [ ]:
preds1.to_csv("predictions3.csv", encoding='utf-8',index=False)

In [ ]:

torch.save(model.state_dict(), "state_dicts.pt")